# House price

---

Le but de ce notebook est d'étudier un jeu de données de maison pour ainsi deviner le prix des maisons à partir de leur descriptif. Le jeu de données est accompagné d'un descriptif des variables. Si jamais vous n'arrivez pas à le récupérer sur git, il est aussi disponible à [cette adresse](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

In [ ]:
# Import des bibliothèques pertinentes

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from xgboost import XGBRegressor

# Partie 1 : analyse de la qualité des données

1) Importer les données. Combien de lignes possède-t-on ? Combien de colonnes ?
2) Bon sang ! ça fait beaucoup de colonnes ! Parmi les colonnes, combien possèdent moins de 10 % de données non nulles ? Ce chiffre vous étonne-t-il ?
2) Quelles sont les 10 colonnes avec le plus de variables manquantes ?
3) Parmi les colonnes, combien possèdent 90 % de valeurs identiques ? Ce chiffre vous étonne-t-il ?
4) Y a-t-il des doublons dans le jeu de données ?

In [ ]:
# Récupération des données
train_house = pd.read_csv("house_data/train.csv")

In [ ]:
# Question 2)
print(f"Notre dataframe contient {len(train_house)} lignes et {len(train_house.columns)} colonnes.")

In [ ]:
train_house.head()

In [ ]:
train_house.info()

In [ ]:
train_house.describe()

In [ ]:
# Question 2)
print(f"Il y a {sum(train_house.isna().sum() > (len(train_house) * 0.9))} colonnes avec plus de 90 % de données vides.")

In [ ]:
# Afin de répondre à la question, on essaye d'identifier les trois colonnes avec des données manquantes.
train_house.isna().sum()[train_house.isna().sum() > (len(train_house) * 0.9)]
# On se rend compte qu'il s'agit de colonnes en rapport avec des détails "de luxe" (piscines et Allée).
# La plupart des maisons ne doivent pas en avoir.

In [ ]:
train_house.groupby(["Street"]).count()["Id"].max()

In [ ]:
# Question 3) On possède un petit dataframe. On peut faire un groupby et compter le nombre de valeurs égales.
# Note : pour les plus grosses bases de données, on cherchera des méthodes plus intelligentes (regarder la médiane et les quartiles)
list_col_identical = []
for my_col in train_house.columns:
    if my_col != "Id":
        if train_house.groupby([my_col]).count()["Id"].max() >= (len(train_house) * 0.9):
            list_col_identical.append(my_col)

In [ ]:
print(f"On possède {len(list_col_identical)} colonnes avec 90 % des valeurs identiques au moins. Ces colonnes sont les suivantes : {list_col_identical}.")

# Partie 2) Compréhension du prix des maisons.
1) Tracez la distribution du prix des maisons. Que remarque-t-on ?
2) De façon intuitive, le prix dépend de la taille. Tracez le prix des maisons en fonction de leur taille. Y a-t-il une relation linéaire ? Si oui, tracez sur votre graphe la droite.
3) Identifiez les outliers de la question d'avant.
4) Quelles sont les variables les plus corrélées avec le prix des maisons ?
5) Certaines variables semblent redondantes. Quelles sont les variables avec une très forte corrélation entre elles ?
5) Trouvez une visualisation pertinente qui relie le lien entre le prix des maisons et leur note globale.
6) Les maisons récentes sont-elles plus chères que les maisons anciennes ?

In [ ]:
# Question 1)
# Première constatation : les prix sont répartis avec une distribution qui ressemble à la distribution exponentielle.
# Constatation rassurante, on ne possède aucune maison avec des prix nuls ou négatifs.

fig = px.histogram(train_house, x="SalePrice", marginal="box", title = "Nombre de maisons par prix")
fig.add_vline(x=train_house["SalePrice"].mean(), line_width=3, line_dash="dash", line_color="red")
fig.add_vline(x=train_house["SalePrice"].median(), line_width=3, line_dash="dash", line_color="green")
fig.show()

In [ ]:
# Première constatation : Afin d'avoir un peu de lisibilité, on applique un log à la valeur.
# On constate que le résultat est plus immédiatement séparable.
train_house["logPrice"] = train_house["SalePrice"].apply(lambda x : np.log10(x+1))
fig = px.histogram(train_house, x="logPrice", marginal="box", title = "Nombre de maisons par prix (log de prix utilisé)")
fig.add_vline(x=train_house["logPrice"].mean(), line_width=3, line_dash="dash", line_color="red")
fig.add_vline(x=train_house["logPrice"].median(), line_width=3, line_dash="dash", line_color="green")
fig.show()

In [ ]:
col_taille = [my_col for my_col in train_house.columns if "Area" in my_col]
matrix = train_house[col_taille].corr().round(2)
sns.heatmap(matrix, annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')

In [ ]:
# On constate qu'on possède 5 colonnes de tailles.
# On veut traver avec plotly les graphes et rajouter une droite
# Question : que tracer en X et que tracer en y et pourquoi ?

for my_col in col_taille:
    fig = px.scatter(train_house, x=my_col, y="SalePrice", title=f"Prix de la maison en fonction de {my_col}")
    fig.show()

In [ ]:
# On ne va pas s'intéresser à PoolArea et de MasVnrArea
col_interest = ["GarageArea", "GrLivArea", "LotArea"]

for my_col in col_interest:
    fig = go.Figure()
    coef_directeur = (train_house.loc[train_house[my_col] > 0]["SalePrice"] / train_house.loc[train_house[my_col] > 0][my_col]).mean()
    fig.add_trace(go.Scatter(x=train_house[my_col], y=train_house["SalePrice"], name=f"Prix de la maison en fonction de {my_col}", mode="markers"))
    fig.add_trace(go.Scatter(x=[k for k in range(0, int(max(train_house["SalePrice"])//coef_directeur), 100)], y=[k * coef_directeur for k in range(0, int(max(train_house["SalePrice"])//coef_directeur), 100)], name="Approximation linéaire",mode="lines"))
    fig.update_layout(title=f"Prix de la maison en fonction de {my_col}", xaxis_title=f"Taille de {my_col}", yaxis_title="Prix de la maison")
    fig.show()

In [ ]:
# Question 3) Trouvons les outliers de la question d'avant.
# Notamment les LotArea plus grands que 50k sont louches
train_house.loc[train_house["LotArea"] > 50000]

In [ ]:
# Cependant GrLivArea semble plus pertinent comme variable (valeurs plus serrées). Les deux outliers sont :
train_house.loc[(train_house["GrLivArea"] > 4000) & (train_house["SalePrice"] < 200000)]

In [ ]:
# On constate d'ailleurs que notre prix va beaucoup dépendre de l'environnement de notre maison.
sale_area = train_house.groupby("MSZoning")[["LotArea", "SalePrice"]].mean()
sale_area["square_feet_price"] = sale_area["SalePrice"] / sale_area["LotArea"]
sale_area

In [ ]:
# On constate d'ailleurs que notre prix va beaucoup dépendre de l'environnement de notre maison.
sale_type = train_house.groupby("SaleType")[["LotArea", "SalePrice"]].mean()
sale_type["square_feet_price"] = sale_type["SalePrice"] / sale_type["LotArea"]
sale_type

In [ ]:
# On constate d'ailleurs que notre prix va beaucoup dépendre de l'environnement de notre maison.
sale_condition = train_house.groupby("SaleCondition")[["LotArea", "SalePrice"]].mean()
sale_condition["square_feet_price"] = sale_condition["SalePrice"] / sale_condition["LotArea"]
sale_condition

In [ ]:
# Note : on a souvent des valeurs en doublon sur les colonnes condition et type. Sans doute faut-il simplifier cela.

In [ ]:
# Question 4) Trouvons les variables les plus corrélées ?
# On va calculer la matrice de corrélation et ensuite on classe par corrélation.
# Note : on classe par valeur absolue parce que une corrélation négative est aussi importante qu'une corrélation positive.

corr_val = train_house.corr(numeric_only=True)[["SalePrice"]]
corr_val["abs_corr"] = corr_val["SalePrice"].apply(abs)
sns.heatmap(corr_val.sort_values("abs_corr", ascending=False).head(20), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')

Quel est le problème avec l'approche précédente ? On ignore les variables textuelles. On contate notamment que beaucoup d'entre elles sont des variables hierarchiques. Transformons les et voyons si nous sommes capables de raffiner notre analyse.

In [ ]:
# {'Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'} ==> ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, FireplaceQu, GarageQual, GarageCond
# {'Gd', 'Av', 'Mn', 'No', 'NA'} ==> BsmtExposure
# {'GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'} ==> BsmtFinType1, BsmtFinType2
# {'Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal' } ==> Functional
# {'Fin', 'RFn', 'Unf', 'NA' } ==> GarageFinish
# {'GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'} ==> Fence

# Pour chacun des groupes de colonnes, on va faire un mapping d'une valeur de départ à une valeur d'arrivée. On va exploiter la hierarchie des dolonnes.

In [ ]:
# On crée un dictionnaire pour faire le lien entre les valeurs de départ et d'arrivées.

list1_col = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']
map1_col = {'Ex' : 5, 'Gd' : 4, 'TA' : 3, 'Fa' : 2, 'Po' : 1, 'NA' : 0}

list2_col = ['BsmtExposure']
map2_col = {'Gd' : 4, 'Av' : 3, 'Mn' : 2, 'No' : 1, 'NA' : 0}

list3_col = ['BsmtFinType1', 'BsmtFinType2']
map3_col = {'GLQ' : 6, 'ALQ' : 5, 'BLQ' : 4, 'Rec' : 3, 'LwQ' : 2, 'Unf' : 1, 'NA' : 0}

list4_col = ['Functional']
map4_col = {'Typ' : 7, 'Min1' : 6, 'Min2' : 5, 'Mod' : 4, 'Maj1' : 3, 'Maj2' : 2, 'Sev' : 1, 'Sal' : 0}

list5_col = ['GarageFinish']
map5_col = {'Fin' : 3, 'RFn' : 2, 'Unf' : 1, 'NA' : 0}

list6_col = ['Fence']
map6_col = {'GdPrv' : 4, 'MnPrv' : 3, 'GdWo' : 2, 'MnWw' : 1, 'NA' : 0}

def map_col_to_dict(train_house, col_name, col_map):
    return train_house[col_name].replace(col_map)

In [ ]:
for my_col in list1_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map1_col)

for my_col in list2_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map2_col)

for my_col in list3_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map3_col)

for my_col in list4_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map4_col)

for my_col in list5_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map5_col)

for my_col in list6_col:
    train_house[my_col] = map_col_to_dict(train_house, my_col, map6_col)

In [ ]:
# Deuxième matrice de corrélation. On se rend compte que certaines des variables évoquées plus tôt apparaissent.

corr_val = train_house.corr(numeric_only=True)[["SalePrice"]]
corr_val["abs_corr"] = corr_val["SalePrice"].apply(abs)
sns.heatmap(corr_val.sort_values("abs_corr", ascending=False).head(20), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')

In [ ]:
# Question 5) On va obtenir une immense matrice. C'est coton.

corr_val = train_house.corr(numeric_only=True).round(2)
sns.set(rc={"figure.figsize":(16, 16)})
sns.heatmap(corr_val, vmax=1, vmin=-1, center=0, cmap='vlag', annot=False)

In [ ]:
# Pour regarder les corrélations, il faut aller colonne par colonne et isoler les valeurs au dela d'un seuil.

In [ ]:
# Question 6)
# Quelle colonne mettre en y ? Laquelle en X ?
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_house.sort_values("OverallQual")["OverallQual"].unique(), y=train_house.sort_values("OverallQual").groupby("OverallQual")["SalePrice"].median(), name=f"Médiane du prix de vente des maisons par rapport à leur évaluation", mode="markers+lines"))
fig.add_trace(go.Scatter(x=train_house.sort_values("OverallQual")["OverallQual"].unique(), y=train_house.sort_values("OverallQual").groupby("OverallQual")["SalePrice"].quantile(0.25), name=f"Premier quartile du prix de vente des maisons par rapport à leur évaluation", mode="lines"))
fig.add_trace(go.Scatter(x=train_house.sort_values("OverallQual")["OverallQual"].unique(), y=train_house.sort_values("OverallQual").groupby("OverallQual")["SalePrice"].quantile(0.75), name=f"Troisième quartile du prix de vente des maisons par rapport à leur évaluation", mode="lines"))
fig.update_layout(title=f"Prix de la maison en fonction de leur évaluation. Premier et troisième quartile", xaxis_title=f"Evaluation", yaxis_title="Prix de la maison")
fig.show()

In [ ]:
# Question 7) On regarde l'année de vente. Il n'y a pas de lien direct
fig = px.histogram(train_house.sort_values("YrSold"), x="SalePrice", marginal="box", color="YrSold", title = "Répartition des maisons en fonction de leur année de vente", barmode="overlay", opacity=0.75)
fig.show()

In [ ]:
# Question 7) Les box plots sont pratiques mais un poil mystérieux. Est-ce qu'il est possible de faire quelque chose de plus simple ?
fig = px.box(train_house.sort_values("YrSold"), x="YrSold", y="SalePrice", color="YrSold", title = "Répartition des prix des maisons par années de vente")
fig.show()

In [ ]:
# Question 7) Les box plots sont pratiques mais un poil mystérieux. Est-ce qu'il est possible de faire quelque chose de plus simple ?
fig = px.box(train_house.sort_values("YearBuilt"), x="YearBuilt", y="SalePrice", color="YearBuilt", title = "Répartition des prix des maisons par années de construction")
fig.show()

In [ ]:
# La médiane et les quartiles donnent déjà beaucoup d'informations. Utilisons-là
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_house.sort_values("YearBuilt")["YearBuilt"].unique(), y=train_house.sort_values("YearBuilt").groupby("YearBuilt")["SalePrice"].median(), name=f"Médiane du prix de vente des maisons par années", mode="markers+lines"))
fig.add_trace(go.Scatter(x=train_house.sort_values("YearBuilt")["YearBuilt"].unique(), y=train_house.sort_values("YearBuilt").groupby("YearBuilt")["SalePrice"].quantile(0.1), name=f"1er décile du prix de vente des maisons par années", mode="lines"))
fig.add_trace(go.Scatter(x=train_house.sort_values("YearBuilt")["YearBuilt"].unique(), y=train_house.sort_values("YearBuilt").groupby("YearBuilt")["SalePrice"].quantile(0.9), name=f"9ème décile du prix de vente des maisons par années", mode="lines"))
fig.update_layout(title=f"Prix de la maison en fonction de l'année de construction. Premier et troisième quartile", xaxis_title=f"Année de construction", yaxis_title="Prix de la maison")
fig.show()

Récapitulatif de ce que l'on sait.

Relation linéaire entre taille de la maison et son prix. Great. Relation avec la qualité de la maison. Pas de relation préciser avec année de vente. Relation avec année de construction. Pas mal de colonnes vides et inutiles + redondances

# Partie 3) Préparons la partie machine learning
1) Au vue de la partie 1, quelles colonnes vous semblent pertinentes à ôter de notre dataframe ?
2) Que faire des outliers identifiés en partie 2 ?
2) Gérez les colonnes avec des valeurs manquantes.
2) Y a-t-il des variables qui mériteraient d'être transformées ? (par exemple, on possède 4 variables en rapport avec les salles de bain)
2) On constate que certaines de nos données catégoriques possèdent une hierarchie. Catégorisez-les de façon logique.
3) Catégorisez le reste des données puis normalisez les. (Note : supprimez les catégories trop rares).
3) Faites une régression linéaire. Visualisez les coefficients pour les différentes colonnes. Quelles sont les colonnes avec les plus forts coefficients ? Cela vous semble-t-il logique ?
3) Faites un ridge regression avec différentes valeurs de Alpha. Visualisez les coefficients pour les différentes colonnes. Quelles sont les colonnes avec les plus forts coefficients ? Cela vous semble-t-il logique ?
3) Essayez de faire une prédiction avec un arbre de décision (DecisionTreeRegressor). Ce modèle vous emble-t-il adapté à notre problème ?
3) Il est temps de dévoiler notre puissance ! Utilisez un XGBoost et plions ce problème comme la crêpe insignifiante qu'il a toujours été !

In [ ]:
# Faisons les choses dans l'autre sens. Quelles sont les colonnes à conserver ?

# MsSubClass a l'air intéressante mais c'est une variable catégorique et pas numérique
train_house["MSSubClass"] = train_house["MSSubClass"].fillna("unknown").apply(str)

# On possède année construction et rénovation. On conserve année la plus récente entre les deux
train_house["YearBuilt"] = train_house[["YearBuilt","YearRemodAdd"]].max(axis=1)

# Tous ont un effet positif ou nul sur le prix ==> On fusionne pour simplifier notre problème
train_house["NbBathroom"] = train_house["BsmtFullBath"] + train_house["BsmtHalfBath"] / 2 + train_house["FullBath"] + train_house["HalfBath"] / 2

# On ne fusionne pas les tailles de porches car tous pas même effet sur le prix
#train_house["PorchArea"] = train_house["OpenPorchSF"] + train_house["EnclosedPorch"] + train_house["3SsnPorch"] + train_house["ScreenPorch"]

# Est-ce que c'est bien d'ajouter YearBuilt alors que l'année est vouée à changer dans le temps ? ==> Valeur évolue lentement. On réentrainera notre modèle tous les ans et ça passe.
# GarageCars doit dépendre de GarageArea. On supprime GarageArea (on choisit par rapport coef corr avec le salePrice)
# BsmtFinSF et BsmtFinType ont une forte corrélation l'un avec l'autre ==> On en conserve un et pas l'autre (on choisit par rapport coef corr avec le salePrice)
col_train = ["MSSubClass", "MSZoning", "LotFrontage", "LotShape", "LandContour", "LotConfig", "Neighborhood", 
       "Condition1", "BldgType", 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinSF2',
       'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageQual',
       'WoodDeckSF', 'Fence', 'SaleType', 'SaleCondition', 'SalePrice', "NbBathroom"]

# On pourrait sans doute faire plus de modifications mais on fait avec ça et c'est déjà bien.

In [ ]:
# Question 2) Nous comptons enlever les deux outliers.
id_to_delete = train_house.loc[(train_house["GrLivArea"] > 4000) & (train_house["SalePrice"] < 200000)]["Id"].values
train_house_no_outlier = train_house.loc[~train_house["Id"].isin(id_to_delete)]

In [ ]:
# On conserve uniquement les colonnes qui nous intéressent.
train_house_filtered = train_house_no_outlier[col_train]

In [ ]:
# Question 3) Manière simple : on associe la médiane aux valeurs numériques et on met une nouvelle catégorie aux valeurs catégoriques.
# On fait la différence entre les deux types de colonnes en regardant leur dtypes.
# Les questions 4 et 5 ont été traitées au fur et à mesure du notebook.
for my_col in train_house_filtered.columns:
    if train_house_filtered[my_col].dtypes == 'O':
        train_house_filtered.loc[train_house_filtered[my_col].isna(), my_col] = "NA"
    else:
        train_house_filtered.loc[train_house_filtered[my_col].isna(), my_col] = train_house_filtered[my_col].median()

In [ ]:
# On vérifie qu'il n'y ait plus de valeurs vides
train_house_filtered.isna().sum()

In [ ]:
# Question 6)
# On va utiliser la méthode pd.get_dummies
train_house_with_dummies = pd.get_dummies(train_house_filtered)
print(f"Notre nouveau dataframe avec des dummies contient {len(train_house_with_dummies.columns)} colonnes. C'est beaucoup.")

In [ ]:
# Dans un cas plus poussé, on étudierait les catégories plus en détails afin de pouvoir faire le tri dans les valeurs.
# Dans notre cas, on va supprimer les colonnes pour lesquelles on possède moins de 10 % de valeurs non nulles.
col_keep_dummy = []
for my_col in train_house_with_dummies.columns:
    if train_house_with_dummies[my_col].dtypes == 'bool':
        if sum(train_house_with_dummies[my_col]) >= len(train_house_with_dummies) * 0.1:
            col_keep_dummy.append(my_col)
    else:
        col_keep_dummy.append(my_col)
    
train_house_with_dummies_filtered = train_house_with_dummies[col_keep_dummy]

print(f"En enlevant les valeurs très peu représentées, on possède un dataframe avec {len(train_house_with_dummies_filtered.columns)} colonnes. C'est moins.")

In [ ]:
# Autre possibilité : target encoding. On encode par rapport au résultat que l'on attend
# Un des problèmes du target encoding est la fuite d'informations (je donne une information que je ne suis pas sensé connaître).
# Cela ne doit pas poser de problème si notre jeu de données est représentatif.

sale_price_med = train_house_filtered["SalePrice"].median()

for my_col in train_house_filtered.columns:
    if train_house_filtered[my_col].dtypes == 'O':
        map_dict = ((train_house_filtered.groupby(my_col)["SalePrice"].median() - sale_price_med) / sale_price_med).to_dict()
        train_house_filtered[my_col] = map_col_to_dict(train_house_filtered, my_col, map_dict)

In [ ]:
# Question 7) Faire avec et sans log dans regression
# Nous devons comparer One-Hot Encoding, target encoding, avec et sans normalisation et avec et sans log.
# Comme on va commencer à séparer nos valeurs en jeux de données de train et de test, nous commençons par mettre une seed pour permettre à nos expériences d'être reréalisable.

np.random.seed(42)

X_train_linear, X_test_linear, y_train_linear, y_test_linear = train_test_split(train_house_with_dummies_filtered[[my_col for my_col in train_house_with_dummies_filtered.columns if my_col != "SalePrice"]], train_house_with_dummies_filtered[["SalePrice"]], test_size=0.2)
my_reg = LinearRegression().fit(X_train_linear, y_train_linear)

In [ ]:
# Quand on n'a pas normalisé les colonnes, les coefficients de notre regression ne sont pas vraiment interprétables / comparables.
# Ils peuvent aussi être arbitrairement grands (pas normalisation). La régression linéaire n'est pas vraiment la meilleure des idées.
my_reg.coef_

In [ ]:
# On observe les performances de notre modèle.
test_prediction = my_reg.predict(X_test_linear)
y_test_linear.reset_index(drop=True, inplace=True)
y_test_linear[["AbsDifference"]] = (y_test_linear - pd.DataFrame(test_prediction, columns = ["SalePrice"])).apply(abs)

fig = px.box(y_test_linear, x="AbsDifference", title = "Répartition de l'erreur absolue sur les prédictions de prix des maisons")
fig.show()

In [ ]:
# Question 8, on va essayer de faire une ridge regression avec des données normalisées.
# Note : il est sans doute possible de rajouter de la cross validation. Ce sera pour une autre fois. 
np.random.seed(42)
train_house_with_dummies_filtered["LogPrice"] = train_house_with_dummies_filtered["SalePrice"].apply(lambda x : np.log10(x+1))
log_mean = train_house_with_dummies_filtered["LogPrice"].mean()
log_std = train_house_with_dummies_filtered["LogPrice"].std()
for my_col in train_house_with_dummies_filtered.columns:
    if (my_col != "SalePrice"):
        train_house_with_dummies_filtered[my_col] = (train_house_with_dummies_filtered[my_col] - train_house_with_dummies_filtered[my_col].mean())/train_house_with_dummies_filtered[my_col].std()

X_train_ridge, X_test_ridge, y_train_ridge, y_test_ridge = train_test_split(train_house_with_dummies_filtered[[my_col for my_col in train_house_with_dummies_filtered.columns if (my_col != "LogPrice") and (my_col != "SalePrice")]], train_house_with_dummies_filtered[["LogPrice"]], test_size=0.2)

In [ ]:
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 20, 30, 50, 75]
error_train = []
error_test = []
y_train_ridge.reset_index(inplace=True, drop=True)
y_test_ridge.reset_index(inplace=True, drop=True)

for alpha in alphas:
    my_reg = Ridge(alpha=alpha).fit(X_train_ridge, y_train_ridge)
    train_prediction = my_reg.predict(X_train_ridge)
    error_train.append((y_train_ridge - pd.DataFrame(train_prediction, columns = ["LogPrice"])).apply(abs).mean().values[0])
    test_prediction = my_reg.predict(X_test_ridge)
    error_test.append((y_test_ridge - pd.DataFrame(test_prediction, columns = ["LogPrice"])).apply(abs).mean().values[0])

In [ ]:
# 2 constatations : l'erreur de test est minimal pour alpha = 20. Nous allons donc garder cette valeur d'alpha.
# L'erreur de train augmente constamment. Cela semble logique. On rajoute des contraintes à notre problème.

error_dataframe = pd.DataFrame({"alphas" : alphas, "error_train" : error_train, "error_test" : error_test})

fig = go.Figure()
fig.add_trace(go.Scatter(x=error_dataframe["alphas"], y=error_dataframe["error_train"], name=f"Erreur obtenue dans notre jeu d'entrainement", mode="markers+lines"))
fig.add_trace(go.Scatter(x=error_dataframe["alphas"], y=error_dataframe["error_test"], name=f"Erreur obtenue dans notre jeu de test", mode="markers+lines"))
fig.update_layout(title=f"Comparaison des valeurs d'erreurs en fonction des valeurs d'alpha", xaxis_title=f"Valeur d'alpha", yaxis_title="Erreur absolue moyenne")
fig.show()

In [ ]:
my_reg = Ridge(alpha=20).fit(X_train_ridge, y_train_ridge)
test_prediction = 10 ** (my_reg.predict(X_test_ridge) * log_std + log_mean)
y_test_linear["AbsDifferenceRidge"] = (y_test_linear[["SalePrice"]] - pd.DataFrame(test_prediction, columns = ["SalePrice"])).apply(abs)

fig = px.box(y_test_linear, x="AbsDifferenceRidge", title = "Répartition de l'erreur absolue sur les prédictions de prix des maisons")
fig.show()

In [ ]:
y_test_linear["AbsPercentRidge"] = (y_test_linear[["SalePrice"]] - pd.DataFrame(test_prediction, columns = ["SalePrice"])).apply(abs) / y_test_linear[["SalePrice"]] *100

fig = px.box(y_test_linear, x="AbsPercentRidge", title = "Répartition de l'erreur absolue en pourcentage sur les prédictions de prix des maisons")
fig.show()

In [ ]:
coef_df = pd.DataFrame({"labels" : X_test_ridge.columns, "CoefVal" : my_reg.coef_[0]})
coef_df["AbsCoefVal"] = coef_df["CoefVal"].apply(abs)
coef_df.set_index("labels", inplace=True)
sns.heatmap(coef_df.sort_values("AbsCoefVal", ascending=False).head(20), annot=True, center=0, cmap='vlag')

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=y_test_linear["AbsDifference"], name="Erreur absolue pour LinearRegression"))
fig.add_trace(go.Box(x=y_test_linear["AbsDifferenceRidge"], name="Erreur absolue pour RidgeRegression"))
fig.update_layout(title=f"Comparaison des valeurs d'erreurs en fonction du modèle de prédiction", xaxis_title=f"Valeur d'erreur")
fig.show()

In [ ]:
# Question 9, on va utiliser un decision tree regressor. Voyons voir ce qui se passe.
np.random.seed(42)

X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(train_house_with_dummies_filtered[[my_col for my_col in train_house_with_dummies_filtered.columns if (my_col != "LogPrice") and (my_col != "SalePrice")]], train_house_with_dummies_filtered[["LogPrice"]], test_size=0.2)

In [ ]:
# On fait du grid search afin de tester les paramètres.
max_depth_list = [i for i in range(2,32)]
error_train = []
error_test = []
y_train_tree.reset_index(inplace=True, drop=True)
y_test_tree.reset_index(inplace=True, drop=True)

for max_depth in max_depth_list:
    my_reg = DecisionTreeRegressor(max_depth=max_depth).fit(X_train_tree, y_train_tree)
    train_prediction = my_reg.predict(X_train_tree)
    error_train.append((y_train_tree - pd.DataFrame(train_prediction, columns = ["LogPrice"])).apply(abs).mean().values[0])
    test_prediction = my_reg.predict(X_test_tree)
    error_test.append((y_test_tree - pd.DataFrame(test_prediction, columns = ["LogPrice"])).apply(abs).mean().values[0])

In [ ]:
# On constate que pour les valeurs les plus basses de max_depth, on a un phénomène d'underfit et pour max_depth trop grand, on a un overfit.
# De façon générale, notre erreur ne baisse pas vraiment ==> On laisse tomber.
# Si vous voulez visualiser l'arbre, cf la dernière partie du Titanic.

error_dataframe = pd.DataFrame({"max_depth" : max_depth_list, "error_train" : error_train, "error_test" : error_test})

fig = go.Figure()
fig.add_trace(go.Scatter(x=error_dataframe["max_depth"], y=error_dataframe["error_train"], name=f"Erreur obtenue dans notre jeu d'entrainement", mode="markers+lines"))
fig.add_trace(go.Scatter(x=error_dataframe["max_depth"], y=error_dataframe["error_test"], name=f"Erreur obtenue dans notre jeu de test", mode="markers+lines"))
fig.update_layout(title=f"Comparaison des valeurs d'erreurs en fonction de la profondeur de notre arbre de décision", xaxis_title=f"max_depth", yaxis_title="Erreur absolue moyenne")
fig.show()

In [ ]:
# Question 10 : XGBoost
# Witness true power.

np.random.seed(42)

X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(train_house_with_dummies_filtered[[my_col for my_col in train_house_with_dummies_filtered.columns if (my_col != "LogPrice") and (my_col != "SalePrice")]], train_house_with_dummies_filtered[["LogPrice"]], test_size=0.2)

In [ ]:
params = {
        'min_child_weight': [1, 2, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb = XGBRegressor(learning_rate=0.02, n_estimators=600, objective='reg:squarederror', nthread=1)

In [ ]:
folds = 3
param_comb = 5

kf = KFold(n_splits=folds, shuffle = False)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=kf.split(X_train_tree, y_train_tree), verbose=3)
random_search.fit(X_train_tree, y_train_tree)

In [ ]:
test_prediction = 10 ** (random_search.best_estimator_.predict(X_test_tree) * log_std + log_mean)
y_test_linear["AbsDifferenceXG"] = (y_test_linear[["SalePrice"]] - pd.DataFrame(test_prediction, columns = ["SalePrice"])).apply(abs)

fig = px.box(y_test_linear, x="AbsDifferenceXG", title = "Répartition de l'erreur absolue sur les prédictions de prix des maisons (XGB)")
fig.show()

Note : on ne fait pas franchement mieux. Le modèle est trop compliqué pour nos données. On va s'arrêter là.
Note : le modèle XGBoost est un modèle boite noire. S'il rend de très bon resultat, je ne sais pas comment il les calcule.
Nous allons voir comment lui donner un peu de clarté grâce à la méthode Shap. (Parler de LIME). On verra ça plus tard.